<a href="https://colab.research.google.com/github/lqiang79/udacity_DSND_arvato/blob/master/Arvato_Project_Workbook_zh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 毕业项目：为 Arvato Financial Services 金融服务公司实现一个顾客分类报告

该项目要求你分析德国的一家邮购公司的顾客的人口统计数据，将它和一般的人口统计数据进行比较。你将使用非监督学习技术来实现顾客分类，识别出哪些人群是这家公司的基础核心用户。之后，你将把所学的知识应用到第三个数据集上，该数据集是该公司的一场邮购活动的营销对象的人口统计数据。用你搭建的模型预测哪些人更可能成为该公司的顾客。你要使用的数据由我们的合作伙伴 Bertelsmann Arvato Analytics 公司提供。这是真实场景下的数据科学任务。

如果你完成了这个纳米学位的第一学期，做过其中的非监督学习项目，那么你应该对这个项目的第一部分很熟悉了。两个数据集版本不同。这个项目中用到的数据集会包括更多的特征，而且没有预先清洗过。你也可以自由选取分析数据的方法，而不用按照既定的步骤。如果你选择完成的是这个项目，请仔细记录你的步骤和决策，因为你主要交付的成果就是一篇博客文章报告你的发现。

In [2]:
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/My Drive/UdacityDataScience/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# magic word for producing visualizations in notebook
%matplotlib inline

§## 第 0 部分：了解数据

项目数据中包括四个数据文件

- `Udacity_AZDIAS_052018.csv`: 德国的一般人口统计数据；891211 人（行）x 366 个特征（列）
- `Udacity_CUSTOMERS_052018.csv`: 邮购公司顾客的人口统计数据；191652 人（行）x 369 个特征（列）
- `Udacity_MAILOUT_052018_TRAIN.csv`: 营销活动的对象的人口统计数据；42982 人（行）x 367 个特征（列）
- `Udacity_MAILOUT_052018_TEST.csv`: 营销活动的对象的人口统计数据；42833 人（行）x 366个特征（列）

人口统计数据的每一行表示是一个单独的人，也包括一些非个人特征，比如他的家庭信息、住房信息以及周边环境信息。使用前两个数据文件中的信息来发现顾客（"CUSTOMERS"）和一般人（"AZDIAS"）在何种程度上相同和不同，然后根据你的分析对其余两个数据文件（"MAILOUT"）进行预测，预测更可能成为该邮购公司的客户。

"CUSTOMERS" 文件包括三个额外的列（'CUSTOMER_GROUP'、’'ONLINE_PURCHASE' 和 'PRODUCT_GROUP'），提供了文件中顾客的更多维度的信息。原始的 "MAILOUT" 包括一个额外的列 "RESPONSE"，表示每个收到邮件的人是否成为了公司的顾客。对于 "TRAIN" 子数据集，该列被保留，但是在 "TEST" 子数据集中该列被删除了，它和你最后要在 Kaggle 比赛上预测的数据集中保留的列是对应的。

三个数据文件中其他的所有列都是相同的。要获得关于文件中列的更多信息，你可以参考 Workspace 中的两个 Excel 电子表格。[其一](./DIAS Information Levels - Attributes 2017.xlsx) 是一个所有属性和描述的列表，按照信息的类别进行排列。[其二](./DIAS Attributes - Values 2017.xlsx) 是一个详细的每个特征的数据值对应关系，按照字母顺序进行排列。

在下面的单元格中，我们提供了一些简单的代码，用于加载进前两个数据集。注意，这个项目中所有的 `.csv` 数据文件都是分号(`;`) 分割的，所以 [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) 中需要加入额外的参数以正确地读取数据。而且，考虑数据集的大小，加载整个数据集可能会花费一些时间。

你会注意到在数据加载的时候，会弹出一个警告（warning）信息。在你开始建模和分析之前，你需要先清洗一下数据。浏览一下数据集的结构，查看电子表格中信息了解数据的取值。决定一下要挑选哪些特征，要舍弃哪些特征，以及是否有些数据格式需要修订。我们建议创建一个做预处理的函数，因为你需要在使用数据训练模型前清洗所有数据集。

In [0]:
# load in the data, This code can only be run on Udacity workspace.
azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

In [63]:
# Be sure to add in a lot more cells (both markdown and code) to document your
# approach and findings!
azdias = pd.read_csv(drive_path+'Udacity_AZDIAS_052018.csv', sep=';')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There is the warning message. we try to find the column current values and check them with possible value in the DIAS attributes.xlsx file. Replace the value with valid one.

In [40]:
print('column 18 label is', azdias.columns[18])
print('column 19 label is', azdias.columns[19])
print(azdias[azdias.columns[18]].unique())
print(azdias[azdias.columns[19]].unique())

column 18 label is CAMEO_DEUG_2015
column 19 label is CAMEO_INTL_2015
[nan '8' '4' '2' '6' '1' '9' '5' '7' '3']
[nan 51.0 24.0 12.0 43.0 54.0 22.0 14.0 13.0 15.0 33.0 41.0 34.0 55.0 25.0
 23.0 31.0 52.0 35.0 45.0 44.0 32.0 '22' '24' '41' '12' '54' '51' '44'
 '35' '23' '25' '14' '34' '52' '55' '31' '32' '15' '13' '43' '33' '45'
 'XX']


In [35]:
azdias['CAMEO_DEUG_2015'] = azdias['CAMEO_DEUG_2015'].map(lambda x: str(x)[0] )
azdias['CAMEO_DEUG_2015'] = azdias['CAMEO_DEUG_2015'].map(lambda x: np.nan if x in ['X', 'n'] else x)
azdias[azdias.columns[18]].unique()

array([nan, '8', '4', '2', '6', '1', '9', '5', '7', '3'], dtype=object)

In [64]:
azdias['CAMEO_INTL_2015'] = azdias['CAMEO_INTL_2015'].astype(str).map(lambda val: np.nan if val in ['nan', 'XX'] else val)
azdias[azdias.columns[19]].unique()

array([nan, '51.0', '24.0', '12.0', '43.0', '54.0', '22.0', '14.0',
       '13.0', '15.0', '33.0', '41.0', '34.0', '55.0', '25.0', '23.0',
       '31.0', '52.0', '35.0', '45.0', '44.0', '32.0', '22', '24', '41',
       '12', '54', '51', '44', '35', '23', '25', '14', '34', '52', '55',
       '31', '32', '15', '13', '43', '33', '45'], dtype=object)

In [78]:
azdias['CAMEO_INTL_2015'] = azdias['CAMEO_INTL_2015'].astype(str).map(lambda x: x[0:1] if x[-2:]=='.0' else x)
azdias[azdias.columns[19]].unique()

array(['nan', '5', '2', '1', '4', '3', '22', '24', '41', '12', '54', '51',
       '44', '35', '23', '25', '14', '34', '52', '55', '31', '32', '15',
       '13', '43', '33', '45'], dtype=object)

In [75]:
dummies = pd.get_dummies(azdias['CAMEO_INTL_2015'], prefix='CAMEO_INTL_2015', prefix_sep='__')
dummies

,CAMEO_INTL_2015__1,CAMEO_INTL_2015__12,CAMEO_INTL_2015__13,CAMEO_INTL_2015__14,CAMEO_INTL_2015__15,CAMEO_INTL_2015__2,CAMEO_INTL_2015__22,CAMEO_INTL_2015__23,CAMEO_INTL_2015__24,CAMEO_INTL_2015__25,CAMEO_INTL_2015__3,CAMEO_INTL_2015__31,CAMEO_INTL_2015__32,CAMEO_INTL_2015__33,CAMEO_INTL_2015__34,CAMEO_INTL_2015__35,CAMEO_INTL_2015__4,CAMEO_INTL_2015__41,CAMEO_INTL_2015__43,CAMEO_INTL_2015__44,CAMEO_INTL_2015__45,CAMEO_INTL_2015__5,CAMEO_INTL_2015__51,CAMEO_INTL_2015__52,CAMEO_INTL_2015__54,CAMEO_INTL_2015__55,CAMEO_INTL_2015__nan
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891216,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
891217,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
891218,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
891219,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [0]:
attributes_df = pd.read_excel(drive_path + 'DIAS Information Levels - Attributes 2017.xlsx', index_col=None, header=1)
values_df = pd.read_excel(drive_path + 'DIAS Attributes - Values 2017.xlsx', index_col=None, header=1)
del attributes_df['Unnamed: 0']
del values_df['Unnamed: 0']

In [96]:
attributes_df.head()

,Information level,Attribute,Description,Additional notes
0,NaN,AGER_TYP,best-ager typology,in cooperation with Kantar TNS; the informatio...
1,Person,ALTERSKATEGORIE_GROB,age through prename analysis,modelled on millions of first name-age-referen...
2,NaN,ANREDE_KZ,gender,NaN
3,NaN,CJT_GESAMTTYP,Customer-Journey-Typology relating to the pref...,"relating to the preferred information, marketi..."
4,NaN,FINANZ_MINIMALIST,financial typology: low financial interest,Gfk-Typology based on a representative househo...


In [114]:
values_df.head(10)

,Attribute,Description,Value,Meaning
0,AGER_TYP,best-ager typology,-1,unknown
1,NaN,NaN,0,no classification possible
2,NaN,NaN,1,passive elderly
3,NaN,NaN,2,cultural elderly
4,NaN,NaN,3,experience-driven elderly
5,ALTERSKATEGORIE_GROB,age classification through prename analysis,"-1, 0",unknown
6,NaN,NaN,1,< 30 years
7,NaN,NaN,2,30 - 45 years
8,NaN,NaN,3,46 - 60 years
9,NaN,NaN,4,> 60 years


In [0]:
values_df['Attribute'].unique()

In [136]:
last_attribute_name = ''
possible_values = set()
attribute_values = {}
for ind, row in values_df.iterrows():
  
  if str(row['Attribute']) != 'nan' and last_attribute_name == '':
    last_attribute_name = row['Attribute']
  possible_values.add(str(row['Value']))

  next_ind = ind + 1
  if next_ind < values_df.shape[0] and str(values_df.iloc[next_ind]['Attribute'])!= 'nan' :
    print(last_attribute_name, possible_values)
    attribute_values[last_attribute_name] = possible_values
    last_attribute_name = ''
    possible_values = set()
  
print(attribute_values)


AGER_TYP {'1', '-1', '0', '2', '3'}
ALTERSKATEGORIE_GROB {'1', '2', '3', '4', '9', '-1, 0'}
ALTER_HH {'1', '20', '17', '14', '8', '0', '13', '10', '12', '11', '16', '7', '18', '4', '21', '9', '5', '19', '2', '3', '6', '15'}
ANREDE_KZ {'2', '1', '-1, 0'}
ANZ_HAUSHALTE_AKTIV {'…'}
ANZ_HH_TITEL {'…'}
ANZ_PERSONEN {'…'}
ANZ_TITEL {'…'}
BALLRAUM {'1', '7', '-1', '2', '3', '6', '4', '5'}
BIP_FLAG {'-1', '1', '0'}
CAMEO_DEUG_2015 {'8', '1', '7', '-1', '2', '3', '6', '4', '9', '5'}
CAMEO_DEU_2015 {'6E', '2A', '6B', '3A', '6C', '2B', '5B', '1E', '1D', '4D', '1A', '9D', '3D', '7D', '9B', '5A', '8D', '7E', '7C', '4E', '9E', '4B', '7A', '1C', '8C', '5E', '6F', '5F', '5D', '9C', '3B', '5C', '2D', '6A', '3C', '4A', '7B', '6D', '9A', '8B', '1B', '4C', '2C', '8A'}
CAMEO_DEUINTL_2015 {'-1', '52', '34', '14', '32', '43', '33', '45', '31', '42', '13', '55', '12', '53', '54', '11', '24', '35', '51', '21', '22', '25', '44', '23', '41', '15'}
CJT_GESAMTTYP {'1', '0', '2', '3', '6', '4', '5'}
D19_BANKEN_ANZ_

In [0]:
dummies = pd.get_dummies(azdias)
dummies

## 第1部分：顾客分类报告

项目报告的主体部分应该就是这部分。在这个部分，你应该使用非监督学习技术来刻画公司已有顾客和德国一般人群的人口统计数据的关系。这部分做完后，你应该能够描述一般人群中的哪一类人更可能是邮购公司的主要核心顾客，哪些人则很可能不是。

## 第2部分：监督学习模型

你现在应该已经发现哪部分人更可能成为邮购公司的顾客了，是时候搭建一个预测模型了。"MAILOUT"数据文件的的每一行表示一个邮购活动的潜在顾客。理想情况下我们应该能够使用每个人的人口统计数据来决定是否该把他作为该活动的营销对象。

"MAILOUT" 数据被分成了两个大致相等的部分，每部分大概有 43 000 行数据。在这部分，你可以用"TRAIN"部分来检验你的模型，该数据集包括一列"RESPONSE"，该列表示该对象是否参加了该公司的邮购活动。在下一部分，你需要在"TEST"数据集上做出预测，该数据集中"RESPONSE" 列也被保留了。

In [0]:
mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

In [8]:
mailout_train = pd.read_csv(drive_path+'Udacity_MAILOUT_052018_TRAIN.csv', sep=';')


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
mailout_train.head

<bound method NDFrame.head of          LNR  AGER_TYP  AKT_DAT_KL  ...  RESPONSE  ANREDE_KZ  ALTERSKATEGORIE_GROB
0       1763         2         1.0  ...         0          2                     4
1       1771         1         4.0  ...         0          2                     3
2       1776         1         1.0  ...         0          1                     4
3       1460         2         1.0  ...         0          2                     4
4       1783         2         1.0  ...         0          1                     3
...      ...       ...         ...  ...       ...        ...                   ...
42957  66338         2         1.0  ...         0          1                     4
42958  67629        -1         1.0  ...         0          1                     3
42959  68273         1         1.0  ...         0          1                     4
42960  68581         2         1.0  ...         0          2                     4
42961  69224         2         1.0  ...         0        

## Part 3:Kaggle比赛

你已经搭建了一个用于预测人们有多大程度上会回应邮购活动的模型，是时候到Kaggle上检验一下这个模型了。如果你点击这个 [链接](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140)，你会进入到比赛界面（如果你已经有一个Kaggle账户的话）如果你表现突出的话，你将有机会收到Arvato或Bertelsmann的人力资源管理的经理的面试邀约！

你比赛用提交的文件格式为CSV，该文件含2列。第一列是"LNR"，是"TEST"部分每个顾客的ID。第二列是"RESPONSE"表示此人有多大程度上会参加该活动，可以是某种度量，不一定是概率。你应该在第2部分已经发现了，该数据集存在一个巨大的输出类不平衡的问题，也就是说大部分人都不会参加该邮购活动。因此，预测目标人群的分类并使用准确率来衡量不是一个合适的性能评估方法。相反地，该项竞赛使用AUC衡量模型的性能。"RESPONSE"列的绝对值并不重要：仅仅表示高的取值可能吸引到更多的实际参与者，即ROC曲线的前端曲线比较平缓。

In [0]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')

```python

```